In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf

## Задание 1

### В данном задании вам предлагается написать несколько SQL-запросов для определенных таблиц. В каждом случае вам НЕ будет предоставлена таблица в явном виде. Однако, вам будет дан пример таблицы с данными.

1. Посчитать % изменение количества клиентов, совершивших покупку, месяц-к-месяцу.

In [2]:
'''Поскольку у нас нет доступа к бд, сгенерируем данные сами, чтобы быть уверенными в корректности
работы запросов. Создадим небольшую таблицу на 1000 строк. ID клиентов сгенерим меньше, чтобы они
повторялись, ID заказов сделаем уникальными, даты в пределах года.'''

np.random.seed(42)
df_1_task = pd.DataFrame()
df_1_task['client_id'] = pd.Series(range(1,800)).sample(1000, replace = True).values
df_1_task['order_id'] = pd.Series(range(10000,99999)).sample(1000).values
df_1_task['order_date'] = pd.Series(pd.date_range(start = '2021-01-01', end = '2021-12-31')) \
                        .sample(1000, replace = True).values
df_1_task.head()

,client_id,order_id,order_date
0,103,78227,2021-10-21
1,436,40913,2021-08-21
2,271,62166,2021-02-07
3,107,15510,2021-10-11
4,72,31922,2021-09-08


Логика работы запроса следующая: сначала из каждой даты "вытащим" месяц года. Для каждого месяца
посчитаем MAU и с помощью оконной функции LAG создадим дополнительную колонку с MAU предыдущего месяца.
Затем просто по формуле посчитаем процентное изменение MAU месяц-к-месяцу.

In [4]:
q = '''WITH mau AS (SELECT strftime('%Y-%m', order_date) as month,
       COUNT(DISTINCT client_id) as mau,
       LAG(COUNT(DISTINCT client_id)) OVER(ORDER BY strftime('%Y-%m', order_date)) as prev_mau
       FROM df_1_task
       GROUP BY month
       ORDER BY month)
       
       SELECT month, mau, (mau - prev_mau) * 100/prev_mau as diff
       FROM mau
       '''

In [5]:
sqldf(q)

,month,mau,diff
0,2021-01,86,NaN
1,2021-02,61,-29.0
2,2021-03,81,32.0
3,2021-04,91,12.0
4,2021-05,85,-6.0
5,2021-06,79,-7.0
6,2021-07,81,2.0
7,2021-08,88,8.0
8,2021-09,67,-23.0
9,2021-10,78,16.0


2. Вывести сумму GMV (Gross Merchandise Value) с нарастающим итогом по дням

In [6]:
'''Здесь тоже создадим таблицу на 100 строк, даты в пределах года,
GMV - просто случайные семизначные числа.'''

np.random.seed(42)
df_2_task = pd.DataFrame()
df_2_task['fact_date'] = pd.Series(pd.date_range(start = '2021-01-01', end = '2021-12-31')) \
                        .sample(100).values
df_2_task['gmv'] = pd.Series(range(1000000,9999999)).sample(100).values
df_2_task.head()

,fact_date,gmv
0,2021-07-13,8619184
1,2021-02-03,4047172
2,2021-01-16,8159935
3,2021-11-06,9819865
4,2021-02-27,3616657


In [7]:
'''Здесь просто с помощью оконной функции считаем накопленную сумму.'''

q = '''SELECT fact_date, gmv,
       SUM(gmv) OVER(ORDER BY fact_date) as cum_sum
       FROM df_2_task
       ORDER BY fact_date
       LIMIT 10'''

In [8]:
sqldf(q)

,fact_date,gmv,cum_sum
0,2021-01-01 00:00:00.000000,9241841,9241841
1,2021-01-04 00:00:00.000000,2013029,11254870
2,2021-01-06 00:00:00.000000,7886690,19141560
3,2021-01-08 00:00:00.000000,5438061,24579621
4,2021-01-10 00:00:00.000000,4139788,28719409
5,2021-01-16 00:00:00.000000,8159935,36879344
6,2021-01-17 00:00:00.000000,3312193,40191537
7,2021-01-18 00:00:00.000000,7203611,47395148
8,2021-01-23 00:00:00.000000,9706757,57101905
9,2021-01-25 00:00:00.000000,2537656,59639561


In [9]:
'''В качестве challenge можно решить без оконки.'''

q = '''SELECT a.fact_date, a.gmv,
       SUM(b.gmv) as cum_sum
       FROM df_2_task a 
       INNER JOIN df_2_task b on a.fact_date>=b.fact_date
       GROUP BY a.fact_date, a.gmv
       ORDER BY a.fact_date'''

In [10]:
'''Видно, что результат полностью идентичен.'''

sqldf(q)

,fact_date,gmv,cum_sum
0,2021-01-01 00:00:00.000000,9241841,9241841
1,2021-01-04 00:00:00.000000,2013029,11254870
2,2021-01-06 00:00:00.000000,7886690,19141560
3,2021-01-08 00:00:00.000000,5438061,24579621
4,2021-01-10 00:00:00.000000,4139788,28719409
...,...,...,...
95,2021-12-16 00:00:00.000000,9383226,526462036
96,2021-12-19 00:00:00.000000,2205077,528667113
97,2021-12-20 00:00:00.000000,1043414,529710527
98,2021-12-26 00:00:00.000000,1024713,530735240


In [11]:
'''Всё верно, общая сумма по столбцу gmv нашего датафрейма равна итоговой накопленной сумме.'''

df_2_task['gmv'].sum()

540528377

3. Получить время отклика на каждое письмо (письмо идентифицируется по полю mail_id), отправленное пользователем mr_employee@ozon.ru.


In [12]:
'''Здесь просто смоделируем небольшую таблицу по предложенному примеру в задании. Добавим еще крайние случаи:
есть отправленное письмо без ответа, есть полученное письмо без ответа, есть цепочка писем между двумя
адресатами с одним и тем же subject.'''

df_3_task = pd.DataFrame()
df_3_task['mail_id'] = range(1, 9)
df_3_task['mail_from'] = ['mr_employee@ozon.ru', 'ms_intern@ozon.ru', 'mr_employee@ozon.ru', 'mr_boss@ozon.ru', 
                          'stranger@ozon.ru', 'mr_employee@ozon.ru', 'ms_intern@ozon.ru', 'mr_employee@ozon.ru']
df_3_task['mail_to'] = ['ms_intern@ozon.ru', 'mr_employee@ozon.ru', 'mr_boss@ozon.ru', 'mr_employee@ozon.ru',
                         'mr_employee@ozon.ru', 'ms_intern@ozon.ru', 'mr_employee@ozon.ru', 'bublik@ozon.ru']
df_3_task['mail_subject'] = ['Задание для практики', 'Задание для практики', 'Отчет по продажам 2021-01-10',
                            'Отчет по продажам 2021-01-10', 'Хотите заработать денег?', 'Задание для практики',
                            'Задание для практики', 'Куда идем обедать?']
df_3_task['timestamp'] = ['2021-01-08 12:00:03', '2021-01-10 13:41:34', '2021-01-11 15:02:57',
                          '2021-01-18 11:03:08', '2021-01-18 12:16:44', '2021-01-20 19:48:54', '2021-01-20 19:52:54',
                          '2021-01-23 11:03:08']
df_3_task['timestamp'] = pd.to_datetime(df_3_task['timestamp'])
df_3_task

,mail_id,mail_from,mail_to,mail_subject,timestamp
0,1,mr_employee@ozon.ru,ms_intern@ozon.ru,Задание для практики,2021-01-08 12:00:03
1,2,ms_intern@ozon.ru,mr_employee@ozon.ru,Задание для практики,2021-01-10 13:41:34
2,3,mr_employee@ozon.ru,mr_boss@ozon.ru,Отчет по продажам 2021-01-10,2021-01-11 15:02:57
3,4,mr_boss@ozon.ru,mr_employee@ozon.ru,Отчет по продажам 2021-01-10,2021-01-18 11:03:08
4,5,stranger@ozon.ru,mr_employee@ozon.ru,Хотите заработать денег?,2021-01-18 12:16:44
5,6,mr_employee@ozon.ru,ms_intern@ozon.ru,Задание для практики,2021-01-20 19:48:54
6,7,ms_intern@ozon.ru,mr_employee@ozon.ru,Задание для практики,2021-01-20 19:52:54
7,8,mr_employee@ozon.ru,bublik@ozon.ru,Куда идем обедать?,2021-01-23 11:03:08


Логика запроса следующая: сджойним таблицу саму на себя таким образом, чтобы к каждому письму, отправленному 
с mr_employee@ozon.ru некоторому адресату, приджойнились все письма, полученные от этого адресата, 
имеющие ту же самую тему и и датированные временем позже, чем дата отправки. Поскольку в цепочке писем может быть несколько,
из всех дат мы выберем минимальную и из нее вычтем дату отправки. Таким образом будет получено время ответа.
Поскольку в задании не указано, в чем измерять результат, будем считать в часах.

In [14]:
q = '''SELECT outgoing.mail_id, (julianday(MIN(incoming.timestamp)) - julianday(outgoing.timestamp))*24 as respond_time
       FROM df_3_task outgoing
       JOIN df_3_task incoming
       ON incoming.mail_subject = outgoing.mail_subject
       AND outgoing.mail_to = incoming.mail_from
       AND incoming.mail_to = outgoing.mail_from
       AND outgoing.timestamp < incoming.timestamp
       WHERE outgoing.mail_from = 'mr_employee@ozon.ru'
       GROUP BY outgoing.mail_id
       '''

In [15]:
sqldf(q)

,mail_id,respond_time
0,1,49.691944
1,3,164.003056
2,6,0.066667


4. Вывести id сотрудников с разницей в заработной плате в пределах 5000 рублей.

In [16]:
'''Сгенерируем датафрейм на 10 строк с id сотрудников от 1 до 10 и рандомными зарплатами в пределах
60 - 100 тыс.руб. Возьмем небольшой разброс, чтобы точно получить нужную разницу между
зарплатами.'''

np.random.seed(42)
df_4_task = pd.DataFrame()
df_4_task['employee_id'] = range(1, 11)
df_4_task['salary_rub'] = pd.Series(range(60000,100000)).sample(10).values
df_4_task

,employee_id,salary_rub
0,1,92823
1,2,76298
2,3,88505
3,4,66689
4,5,86893
5,6,96572
6,7,72335
7,8,89591
8,9,78948
9,10,91067


Здесь просто сделаем CROSS JOIN таблицы самой на себя. Таким образом получим таблицу, 
где напротив id и зарплаты каждого сотрудника будут id и зарплаты всех
его коллег. Затем просто остается посчитать разницу между зарплатами
и оставить только те строки, где разница находится в пределах 5 тыс.руб. в обе стороны и id не равны друг другу.

In [18]:
'''По моему мнению, так таблица выглядит инфоомативнее. Можно увидеть id сотрудников,
их зарплаты и полученную разницу.'''

q = '''SELECT a.employee_id as employee_id_1, b.employee_id as employee_id_2, a.salary_rub as salary_empl_1,
       b.salary_rub as salary_empl_2, a.salary_rub-b.salary_rub as diff
       FROM df_4_task a
       JOIN df_4_task b
       WHERE a.employee_id != b.employee_id 
       AND diff BETWEEN -5000 AND 5000
'''

In [19]:
sqldf(q)

,employee_id_1,employee_id_2,salary_empl_1,salary_empl_2,diff
0,1,3,92823,88505,4318
1,1,6,92823,96572,-3749
2,1,8,92823,89591,3232
3,1,10,92823,91067,1756
4,2,7,76298,72335,3963
5,2,9,76298,78948,-2650
6,3,1,88505,92823,-4318
7,3,5,88505,86893,1612
8,3,8,88505,89591,-1086
9,3,10,88505,91067,-2562


In [20]:
'''Но в задании просят вывести только id, поэтому запрос может выглядеть так:'''

q = '''WITH sal AS (SELECT a.employee_id as employee_id_1, b.employee_id as employee_id_2, a.salary_rub as salary_empl_1,
       b.salary_rub as salary_empl_2, a.salary_rub-b.salary_rub as diff
       FROM df_4_task a
       JOIN df_4_task b
       WHERE a.employee_id != b.employee_id 
       AND diff BETWEEN -5000 AND 5000)
       
       SELECT employee_id_1, employee_id_2
       FROM sal
'''

In [21]:
sqldf(q)

,employee_id_1,employee_id_2
0,1,3
1,1,6
2,1,8
3,1,10
4,2,7
5,2,9
6,3,1
7,3,5
8,3,8
9,3,10
